In [13]:
from typing import Any, Union
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

In [14]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score, f1_score, make_scorer
from sklearn.model_selection import *
from sklearn.pipeline import *
from sklearn.compose import *
from sklearn.preprocessing import *
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.ensemble import *
from sklearn.decomposition import *
from sklearn.impute import *
from sklearn.base import BaseEstimator
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.calibration import *

# PREPROCESS

In [15]:
train = pd.read_csv('data/original/train.csv').drop('ID_y', axis=1)
test = pd.read_csv('data/original/test.csv')

In [16]:
train.head()

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,1,0,1,1,0


In [17]:
CAT_UNORDERED_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 'Профессия', 'Статус Курения',
    'Частота пасс кур', 'Алкоголь', 'Время засыпания', 'Время пробуждения'
]
CAT_ORDERED_COLS = ['Образование_поряд', 'Статус Курения_поряд', 'Частота пасс кур_поряд']
BINARY_COLS = [
    'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет',
    'Гепатит', 'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
    'Туберкулез легких ', 'ВИЧ/СПИД', 'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
    'Пассивное курение', 'Сон после обеда', 'Спорт, клубы', 'Религия, клубы',  
    ]
REAL_COLS = [
    'Возраст курения', 'Сигарет в день', 'Возраст алког', 'Время засыпания_поряд', 
    'Время пробуждения_поряд']



In [18]:
TARGETS = [
    'Сердечная недостаточность', 'Стенокардия, ИБС, инфаркт миокарда', 
    'Прочие заболевания сердца', 'ОНМК', 'Артериальная гипертензия']

In [19]:
def generate_cat_col_pairs():
    n = len(CAT_UNORDERED_COLS)
    for i in range(n):
        c1 = CAT_UNORDERED_COLS[i]
        for j in range(i+1, n):
            c2 = CAT_UNORDERED_COLS[j]
            yield c1, c2, f'{c1}_{c2}'

In [20]:
BI_UNORDERED_CAT_COLS = [paired_col for c1, c2, paired_col in generate_cat_col_pairs()]

In [21]:
def add_paired_categories(data: pd.DataFrame) -> pd.DataFrame:
    for c1, c2, paired_col in generate_cat_col_pairs():
        data[paired_col] = data[c1] + ' | ' + data[c2]
    return data

In [22]:
def add_features(data: pd.DataFrame) -> pd.DataFrame:
    # образование как порядковый признак
    data['Образование_поряд'] = data['Образование'].str.slice(0, 1).astype(np.int8)
    data['Статус Курения_поряд'] = (
        data['Статус Курения'].replace({
            'Никогда не курил(а)': 0,
            'Никогда не курил': 0,
            'Бросил(а)': 1,
            'Курит': 2
        }))
    data['Частота пасс кур_поряд'] = (
        data['Частота пасс кур'].replace({
            '1-2 раза в неделю': 0,
            '3-6 раз в неделю': 1,
            'не менее 1 раза в день': 2,
            '2-3 раза в день': 3,
            '4 и более раз в день': 4
        })
    )
    data['Алкоголь_поряд'] = (
        data['Алкоголь'].replace({
            'никогда не употреблял': 0,
            'ранее употреблял': 1,
            'употребляю в настоящее время': 2
        })
    )


    def process_sleep_time(s: pd.Series) -> pd.Series:
        s = pd.to_datetime(s)
        date = pd.Timestamp(s.iloc[0].date())
        mask = s < (date + pd.Timedelta(hours=12))
        s.loc[mask] = s.loc[mask] + pd.Timedelta(days=1)
        s = (s - date) / pd.Timedelta(hours=1)
        return s

    data['Время засыпания_поряд'] = process_sleep_time(data['Время засыпания'])


    def process_wakeup_time(s: pd.Series) -> pd.Series:
        s = pd.to_datetime(s)
        date = pd.Timestamp(s.iloc[0].date())
        return (s - date) / pd.Timedelta(hours=1)

    data['Время пробуждения_поряд'] = process_wakeup_time(data['Время пробуждения'])
    data = add_paired_categories(data)
    return data

In [23]:
def cast_types(data: pd.DataFrame) -> pd.DataFrame:
    data[REAL_COLS] = data[REAL_COLS].astype(np.float32)
    data[BINARY_COLS] = data[BINARY_COLS].astype(np.int8)
    data[CAT_ORDERED_COLS] = data[CAT_ORDERED_COLS].astype(np.float32)
    data[CAT_UNORDERED_COLS] = data[CAT_UNORDERED_COLS].fillna('NA').astype('category')
    data[BI_UNORDERED_CAT_COLS] = data[BI_UNORDERED_CAT_COLS].fillna('NA').astype('category')
    return data

In [ ]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    data = add_features(data)
    data = cast_types(data)
    data = data.set_index('ID')
    return data

In [24]:
train = preprocess(train)
test = preprocess(test)

In [25]:
train.to_pickle('data/prepared/train.pkl')
test.to_pickle('data/prepared/test.pkl')

In [26]:
X_train = train.drop(TARGETS, axis=1)
y_train = train[TARGETS]

## METRICS

In [27]:
def compute_single_col_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    max_metric = float('-inf')
    for tresh in np.unique(y_pred):
        curr_metric = recall_score(y_true, np.where(y_pred >= tresh, 1, 0), average='macro', zero_division=0)
        if curr_metric > max_metric:
            max_metric = curr_metric
    return max_metric

In [28]:
def compute_weird_pred_proba_score(y_true: np.ndarray, y_pred: Union[list, np.ndarray]) -> float:
    if not isinstance(y_true, np.ndarray):
        y_true = y_true.values

    if isinstance(y_pred, list):
        y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])

    avg_metric = 0
    for col in range(y_true.shape[1]):
        max_metric = compute_single_col_score(y_true[:, col], y_pred[:, col])
        avg_metric += max_metric
    avg_metric /= y_true.shape[1]
    return avg_metric

In [29]:
def get_tresholds(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> tuple[float, dict]:
    if isinstance(y_pred, list):
        y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])
        y_pred = pd.DataFrame(data=y_pred, index=y_true.index, columns=y_true.columns)
        
    avg_metric = 0
    tresholds = {}
    for col in y_true.columns:
        max_metric = float('-inf')
        for tresh in y_pred[col].unique():
            curr_metric = recall_score(y_true[col].values, np.where(y_pred[col].values >= tresh, 1, 0), average='macro', zero_division=0)
            if curr_metric > max_metric:
                max_metric = curr_metric
                tresholds[col] = tresh
        avg_metric += max_metric
    avg_metric /= len(y_true.columns)
    print(avg_metric)
    return tresholds

In [30]:
def compute_weird_pred_score(y_true: np.ndarray, y_pred:np.ndarray) -> float:
    if not isinstance(y_true, np.ndarray):
        y_true = y_true.values

    for col in range(y_true.shape[1]):
        curr_metric = recall_score(y_true[col], y_pred, average='macro', zero_division=0)
        avg_metric += curr_metric
    avg_metric /= y_true.shape[1]
    return avg_metric

In [31]:
weird_pred_proba_score = make_scorer(score_func=compute_weird_pred_proba_score, greater_is_better=True, needs_proba=True)
weird_pred_score = make_scorer(score_func=compute_weird_pred_score, greater_is_better=True, needs_proba=False)
weird_single_col_pred_proba_score = make_scorer(score_func=compute_single_col_score, greater_is_better=True, needs_proba=True)

# MODELING

In [32]:
RANDOM_STATE=7

In [33]:
cv_multi = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=RANDOM_STATE)
cv_single = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)

In [34]:
common_preprocess_pipe = ColumnTransformer(transformers=[
    ('cat_unordered_cols', OneHotEncoder(dtype=np.int8, handle_unknown='ignore', sparse=False), CAT_UNORDERED_COLS+BI_UNORDERED_CAT_COLS),
    ('real_cols', make_pipeline(SimpleImputer(), StandardScaler()), REAL_COLS + CAT_ORDERED_COLS),
    ('binary_cols', 'passthrough', BINARY_COLS)
])

In [ ]:
# max_depth=3, 
# iterations=500, 
# cat_features=CAT_UNORDERED_COLS+BI_UNORDERED_CAT_COLS, 
# silent=True,
# grow_policy='Depthwise',
# random_seed=RANDOM_STATE

In [139]:
cat_model = CatBoostClassifier(
    max_depth=3, 
    iterations=500, 
    cat_features=CAT_UNORDERED_COLS+BI_UNORDERED_CAT_COLS, 
    silent=True,
    grow_policy='Depthwise',
    random_seed=RANDOM_STATE)
# cat_model = CalibratedClassifierCV(base_estimator=cat_model, cv=cv_single, n_jobs=-1, ensemble=True)

lgb = LGBMClassifier()
# lgb = CalibratedClassifierCV(base_estimator=lgb, cv=cv_single, n_jobs=-1, ensemble=True)
lgb_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('lgb', lgb)
])

xgb = XGBClassifier()
# xgb = CalibratedClassifierCV(base_estimator=xgb, cv=cv_single, n_jobs=-1, ensemble=True)
xgb_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('xgb', XGBClassifier())
])

log_reg = LogisticRegressionCV()
# log_reg = CalibratedClassifierCV(base_estimator=log_reg, cv=cv_single, n_jobs=-1, ensemble=True)
log_reg_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('log_reg', log_reg)
])

svm = SVC(probability=True)
# svm = CalibratedClassifierCV(base_estimator=svm, cv=cv_single, n_jobs=-1, ensemble=True)
svm_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('svm', svm)
])

knn = base_estimator=KNeighborsClassifier(n_jobs=-1)
# knn = CalibratedClassifierCV(base_estimator=knn, cv=cv_single, n_jobs=-1, ensemble=True)
knn_preprocess_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('SVD',  TruncatedSVD(n_components=20))
])
knn_pipe = Pipeline([
    ('knn_preprocess_pipe', knn_preprocess_pipe),
    ('knn', knn)
])

rf = RandomForestClassifier(max_depth=20, n_jobs=-1)
# rf = CalibratedClassifierCV(base_estimator=rf, cv=cv_single, n_jobs=-1, ensemble=True)
rf_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('rf', rf)
])

In [140]:
# meta_model = MultiOutputClassifier(
#     estimator=StackingClassifier(n_jobs=-1,
#     final_estimator=LogisticRegressionCV(cv=cv_single),
#     passthrough=False,
#     estimators=[
#         ('cat', cat_model),
#         ('log_reg', log_reg_pipe),
#         ('svm', svm_pipe),
#         ('knn', knn_pipe),
#         ('rf', rf_pipe),
#         ('lgb', lgb_pipe),
#         ('xgb', xgb_pipe),
#     ]
#     )
# )

In [141]:
meta_model = MultiOutputClassifier(cat_model)

## CV

In [142]:
# cat - 0.652999
# lgb - 0.638
# xgb - 0.64133
# lr - 0.6396
# svm - 0.66133
# knn - 0.589
# rf - 0.62799

In [143]:
oof_pred_proba = cross_val_predict(
    estimator=meta_model, 
    X=X_train, 
    y=y_train,
    cv=cv_multi,
    method='predict_proba',
    n_jobs=-1)

In [144]:
tresholds = get_tresholds(y_train, oof_pred_proba)

0.671419873116762


In [145]:
# 0.671419873116762

In [146]:
tresholds

{'Сердечная недостаточность': 0.07429260906808771,
 'Стенокардия, ИБС, инфаркт миокарда': 0.10168634882000596,
 'Прочие заболевания сердца': 0.07517271124036444,
 'ОНМК': 0.026918975628716042,
 'Артериальная гипертензия': 0.48898243911689176}

In [147]:
meta_model = meta_model.fit(X_train, y_train)


In [148]:
sample_submission = pd.read_csv('data/original/sample_solution.csv').set_index('ID')

In [149]:
def make_prediction(test: pd.DataFrame, model: BaseEstimator, tresholds: dict, sample_submission: pd.DataFrame) -> pd.DataFrame:
    assert all(test.index == sample_submission.index)
    
    y_pred = model.predict_proba(test)

    y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])
    y_pred = pd.DataFrame(data=y_pred, index=test.index, columns=TARGETS)
    
    for col in TARGETS:
        sample_submission[col] = np.where(y_pred[col].values >= tresholds[col], 1, 0)
        
    return sample_submission


In [150]:
pred_proba_test = make_prediction(test, meta_model, tresholds, sample_submission)

In [151]:
sample_submission

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-001-019-01,1,1,0,1,1
54-002-133-01,1,1,1,1,1
54-001-007-01,1,0,1,1,1
54-102-116-01,0,1,0,0,0
54-502-005-02,1,1,1,1,0
...,...,...,...,...,...
54-102-095-01,1,1,1,1,1
54-102-235-01,1,0,1,1,1
54-502-016-01,1,1,1,1,1


In [152]:
sample_submission.to_csv('submissions/base_ensemble.csv')